In [2]:
import cv2
import os
from enum import Enum
import numpy as np
from scipy.signal import wiener
from skimage.restoration import unsupervised_wiener, richardson_lucy 
from tqdm import tqdm

In [3]:
# Noise & Artifact Removal
# Resize images
# Possibly change all views to a single view

In [4]:
# Flags
class Resize(Enum):
    MIN = 1
    MAX = 2
    MEAN = 3
    FIXED_INTERPOLATED = 4
    FIXED_CROPPED = 5

class Color(Enum):
    GRAYSCALE = 1
    RGB = 2

class NoiseRemoval(Enum):
    NONE = 1
    GAUSSIAN = 2
    MEDIAN = 3
    BILATERAL = 4

class BlurRemoval(Enum):
    NONE = 1
    WIENER_FILTER = 2
    BLIND_DECONVOLUTION = 3
    RICHARDSON_LUCY = 4

class View(Enum): # TODO: Figure out what to do with views
    SINGLE = 1
    ALL = 2

In [7]:
# Constants
DATASET_PATH = 'dataset'
FLOODED_PATH = 'dataset/flooded'
NON_FLOODED_PATH = 'dataset/non-flooded'

In [8]:
path = os.getcwd()

images_paths = []
images = []
labels = []

folder_path = os.path.join(path, FLOODED_PATH)
for image in os.listdir(folder_path):
    image_path = os.path.join(folder_path, image)
    images_paths.append(image_path)
    labels.append(1)

folder_path = os.path.join(path, NON_FLOODED_PATH)
for image in os.listdir(folder_path):
    image_path = os.path.join(folder_path, image)
    images_paths.append(image_path)
    labels.append(0)

print('Dataset size: ', len(images_paths))
print('Flooded images: ', labels.count(1))
print('Non-flooded images: ', labels.count(0))

Dataset size:  922
Flooded images:  461
Non-flooded images:  461


In [9]:
# Noise Removal Techniques
# TODO: Add more noise removal techniques
# TODO: Check if the parameters need to be changed
def noise_removal(image, noise_removal):
    if noise_removal == NoiseRemoval.GAUSSIAN:
        return cv2.GaussianBlur(image, (5, 5), 0)
    elif noise_removal == NoiseRemoval.MEDIAN:
        return cv2.medianBlur(image, 5)
    elif noise_removal == NoiseRemoval.BILATERAL:
        return cv2.bilateralFilter(image, 9, 75, 75)
    else:
        return image

In [10]:
# Blur Removal Techniques
def blur_removal(image, blur_removal, color):
    if blur_removal == BlurRemoval.WIENER_FILTER:
        if color == Color.GRAYSCALE:
            return wiener(image, (5, 5))   
        else:
            red_channel, green_channel, blue_channel = cv2.split(image)
            deblurred_red_channel = wiener(red_channel, (5, 5))
            deblurred_green_channel = wiener(green_channel, (5, 5))
            deblurred_blue_channel = wiener(blue_channel, (5, 5))
            deblurred_image = cv2.merge((deblurred_red_channel,
                             deblurred_green_channel,
                             deblurred_blue_channel))
            deblurred_image = np.clip(deblurred_image, 0, 255)
            deblurred_image = deblurred_image.astype(np.uint8)
            return deblurred_image
    elif blur_removal == BlurRemoval.BLIND_DECONVOLUTION:
        return unsupervised_wiener(image)
    elif blur_removal == BlurRemoval.RICHARDSON_LUCY:
        return richardson_lucy(image, 5)
    else:
        return image

In [13]:
# Main pipeline
def preprocess(image, noise=NoiseRemoval.GAUSSIAN, blur=BlurRemoval.WIENER_FILTER, resize=Resize.MIN, color=Color.GRAYSCALE):
    if color == Color.GRAYSCALE:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    image = noise_removal(image, noise)
    
    # image = blur_removal(image, blur, color)

    return image

In [14]:
preprocess(images_paths[0], noise=NoiseRemoval.NONE, blur=BlurRemoval.WIENER_FILTER, color=Color.RGB)

'd:\\Fourth_Year\\Second_Term\\BD\\Flooding-Detection\\dataset/flooded\\0.jpg'

In [15]:
# Pipeline for all images
for i in range(len(images)):
    images[i] = preprocess(images[i])